### Snow depth prediction with topographic parameters
From the article:
High-resolution snow depth prediction using Random Forest algorithm with topographic parameters: A case study in the Greiner watershed, Nunavut

Meloche et al 2022 
https://doi.org/10.1002/hyp.14546

The first section trains the RF algorithm with topographic parameter. (see csv file)

I suggest you create an env with those library

### Python Library:
- Numpy
- pandas
- seaborn and pyplot (for graph)
- scikit learn for Random forest

for prediction and creating snow depth map in .tiff
- xarray
- rasterio
- rioxarray

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
import rasterio as rio
import rioxarray

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [ ]:
#Define the function that will run scikit Random forest
def randomForest(X, y, training_size=0.5):
    """
    X: array of input variable (predictor)
    y: snow depth for training
    
    return:
    regressor : (obj type) to get feature importance
    y_test, x_test : numpy array, x and y for testing accuracy
    y_pred :  numpy array, y for prediction
    """
    #test size is set to 50% by default, modify to your liking when calling function
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= training_size, random_state = 0)
    print(f'train : {len(y_train)}, test : {len(y_test)}')
    # no difference with standard
    #sc = StandardScaler()
    #X_train = sc.fit_transform(X_train)
    #X_test = sc.transform(X_test)

    regressor = RandomForestRegressor(n_estimators=400, n_jobs = -1, random_state=0)
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)

    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print(f'Explained Variance : {metrics.explained_variance_score(y_test, y_pred)} ')
    print(f'R² : {metrics.r2_score(y_test, y_pred)} ')
    
    return regressor, y_test, y_pred, X_test


In [ ]:
# get (X, y) data for training
#X topo parameter 
#y snow depth from magnaprobe
df_corr = pd.read_csv('magnaprobe_clean_Greiner.csv')
df_corr.columns

# the parameters chosen are the most optimal, user can use more if wanted or play with parameters
X = pd.DataFrame({'Sx' : df_corr.Sx_mean, 'TPI_150m' : df_corr.TPI_mean, 'Slope' : df_corr.slope_mean, 
                  #'Aspect' : df_corr.aspect_mea,
                  #'Elevation' : df_corr.elev_mean, 
                  'Ecotype' : df_corr.Eco_majori, 
                  #'Veg Height' : df_corr.Veg_height,
                  #'NDVI' : df.ndvi_mean,
                  'Year mean' : df_corr.u_sd_temp,
                  #'TPI_500' : df_corr.TPI_500_me,
                  #'TPI_1km' : df_corr.TPI_1000_m,
                  'TPI_5km' : df_corr.TPI_5km_me,
                  #'TPI_10km' : df_corr.TPI_10km_m,
                 })

# get dpeth fro training
y = df_corr.loc[:,"depth"]/100

In [ ]:
#run the training algorithm
#get accuracy
regressor, y_test, y_pred, X_test = randomForest(X,y)
hist_y_corr = regressor.feature_importances_

In [ ]:
# plot feature importance
label = ['Sx', 'TPI', 'slope', 'Ecotype', 'Year mean', 'TPI_5km']
plt.xticks(rotation=90)
plt.bar(label, hist_y_corr)
plt.ylabel('Feature importance', size = 15)
plt.title('Random Forest variable', size = 15)
plt.yticks(size = 12)
plt.xticks(size = 15)

In [ ]:
#Graph validation from RF prediction

#sns.set('paper')
plt.figure(figsize = (12,4))
plt.subplot(121)
plt.title('a)', size = 20)
x = np.linspace(0,1.5,100)
plt.plot(x,x, color = 'black')
plt.xlim(0,1.5)
plt.ylim(0,1.5)
plt.yticks(size = 12)
plt.xticks(size = 12)
plt.ylabel('Snow depth modeled (m)', size = 15)
plt.xlabel('Snow depth measured (m)', size = 15)
plt.scatter(y_test, y_pred, color = 'red', alpha = 0.5)

plt.subplot(122)
plt.title('b)', size = 20)
sns.residplot(x = y_test, y = y_pred)
plt.ylabel('Residuals (m)', size = 15)
plt.xlabel('Snow depth (m)', size =15)
plt.yticks(size = 12)
plt.xticks(size = 12)

## Prepare input for entire watershed
This section is to run the random forest across the full watershed.

The output is in raster format (.tiff), EPSG = 26913 (NAD83 / UTM zone 13N)

X or topographic parameters comes in netcdf format (RF_variable_GreinerWatershed.nc)

for prediction of new snowmap, the mean depth is define by the user

In [ ]:
# average depth per year is use for training
#average depth needs to be provided for prediction
snowdepth_mean = 0.40

In [ ]:
#open dataset of topographic parameter for prediction
#data cube
data = xr.open_dataset('RF_variable_GreinerWatershed.nc')

In [ ]:
#prepare data (X variable) (flatten array for prediction)
tmp_tpi = data.TPI_150m.values.flatten()
tmp_sx = data.Sx_R100_W315.values.flatten()
tmp_slope = data.slope.values.flatten()
tmp_elev = data.elev.values.flatten()
tmp_eco = data.eco.values.flatten()

In [ ]:
#get shape of raster
shape = data.TPI_150m.values.shape
#get index of pixel for correspondace of value predicted by RF
XX,YY = np.meshgrid(np.arange(shape[1]),np.arange(shape[0]))

#remove water value from prediction in RF with ecotype 1,2 and 3
eco = data.eco.values
indexx, indexy = np.where(eco == 1)

In [ ]:
#set array for year mean, same len of other parameter
y_mean = np.ones(len(tmp_sx)) * snowdepth_mean

In [ ]:
#building X and Y variable for prediction
#order of Variable needs to match regressor from training data
X_data = pd.DataFrame({'Sx' : tmp_sx, 'TPI' : tmp_tpi, 'Slope' : tmp_slope,
                       #'Aspect' : tmp_aspect, 
                       'Elevation' : tmp_elev,
                       'Ecotype' : tmp_eco,
                       'Year mean' : y_mean,
                       'XX' : XX.flatten(), 'YY' : YY.flatten()}).dropna()

# get index inforamtion
XX_clean = np.array(X_data.XX)
YY_clean = np.array(X_data.YY)
X_data = X_data.drop(labels = ['XX', 'YY'], axis = 1)

In [ ]:
#predict snow depth 
Y_pred = regressor.predict(X_data)

In [ ]:
# create array with original shape 
arr = np.zeros(shape)
arr.fill(np.nan)
# fill with prediction from Y_pred and corresponding index
for i in np.arange(len(XX_clean)):
    arr[YY_clean[i], XX_clean[i]] = Y_pred[i]

In [ ]:

# set ecotype of water to nan for transprancy in QGIS
indexx, indexy = np.where(eco == 1)
arr[(indexx, indexy)]=np.nan
indexx, indexy = np.where(eco == 2)
arr[(indexx, indexy)]=np.nan
indexx, indexy = np.where(eco == 3)
arr[(indexx, indexy)]=np.nan

#create xarray dataset to write tiff file
snow = xr.DataArray(arr, coords = [data.eco.y.values, data.eco.x.values], dims = [ 'y', 'x'])
#write tiff file
snow.rio.write_crs("epsg:26913", inplace=True)
snow.rio.to_raster("Ecotype_Greiner_snowRF.tif")